In [6]:
import pyomo.environ as pyo

In [7]:
model = pyo.ConcreteModel()

# 변수 정의
model.food = pyo.Set(initialize=['brownie','icecream','cola','cheesecake'])
model.calories = pyo.Param(model.food, initialize={'brownie':400,'icecream':200,'cola':150,'cheesecake':500})
model.choco = pyo.Param(model.food, initialize={'brownie':3,'icecream':2,'cola':0,'cheesecake':0})
model.sugar = pyo.Param(model.food, initialize={'brownie':2,'icecream':2,'cola':4,'cheesecake':4})
model.fat = pyo.Param(model.food, initialize={'brownie':2,'icecream':4,'cola':1,'cheesecake':5})
model.cost = pyo.Param(model.food, initialize={'brownie':50,'icecream':20,'cola':30,'cheesecake':80})

# 변수 정의
model.x = pyo.Var(model.food, within=pyo.NonNegativeReals)

# 목적 함수
model.obj = pyo.Objective(expr=sum(model.cost[i]*model.x[i] for i in model.food))

# 제약 조건
model.con_calories = pyo.Constraint(expr=sum(model.calories[i]*model.x[i] for i in model.food) >= 500)
model.con_choco = pyo.Constraint(expr=sum(model.choco[i]*model.x[i] for i in model.food) >= 6)
model.con_sugar = pyo.Constraint(expr=sum(model.sugar[i]*model.x[i] for i in model.food) >= 10)
model.con_fat = pyo.Constraint(expr=sum(model.fat[i]*model.x[i] for i in model.food) >= 8)

# 모델 해결
model.solver = pyo.SolverFactory('ipopt')

# 결과 출력
# pyomo 계산 결과를 출력하는 코드
result = model.solver.solve(model)
for f in model.food:
    print(f"{f}: {pyo.value(model.x[f])}")

print(f"\n목적함수 값: {pyo.value(model.obj)}")

# 각 제약 조건이 어떻게 만족되었는지 출력
print("\n각 제약 조건의 실제 값:")
print(f"총 칼로리: {sum(pyo.value(model.calories[i]) * pyo.value(model.x[i]) for i in model.food)} (>= 500)")
print(f"총 초콜릿: {sum(pyo.value(model.choco[i]) * pyo.value(model.x[i]) for i in model.food)} (>= 6)")
print(f"총 설탕: {sum(pyo.value(model.sugar[i]) * pyo.value(model.x[i]) for i in model.food)} (>= 10)")
print(f"총 지방: {sum(pyo.value(model.fat[i]) * pyo.value(model.x[i]) for i in model.food)} (>= 8)")


brownie: -9.908874210129835e-09
icecream: 2.999999985364352
cola: 0.9999999973056684
cheesecake: -9.949883563498269e-09

목적함수 값: 89.99999833502271

각 제약 조건의 실제 값:
총 칼로리: 749.9999877302291 (>= 500)
총 초콜릿: 5.9999999410020814 (>= 6)
총 설탕: 9.999999900334096 (>= 10)
총 지방: 12.99999986919591 (>= 8)
